In [1]:
import os
os.chdir("..")

In [2]:
import pandas as pd
import numpy as np

In [69]:
pd.set_option("display.max_rows", None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

# 1. Scrape unlabeled reports

In [52]:
# Load data
df_unlabeled = pd.read_excel('data/interim/reports_unlabeled.xlsx', engine='openpyxl')
df_unlabeled

,Identifier,Company_Name,Country_of_Exchange,CSR_Period_Absolute,CSR_Period_Relative,CSR_Title,CSR_URL
0,BCE_py.TO,BCE Inc,Canada,FY2020,FY0,00 Purpose and Corporate Responsibility Report,https://www.bce.ca/responsibility/overview/2020-cr-report.pdf
1,ENB_pa.TO,Enbridge Inc,Canada,FY2020,FY0,00 Sustainability Report,https://www.enbridge.com/~/media/Enb/Documents/Reports/Sustainability%20Report%202020/Enbridge_SR_2020.pdf
...,...,...,...,...,...,...,...
15098,FLTRF.L,Flutter Entertainment PLC,United Kingdom,FY2011,FY-9,Annual Report 0,http://www.paddypowerplc.com/uploads/documents/annualreport2011.pdf
15099,TUIT.L,TUI AG,United Kingdom,FY2010,FY-9,Annual Report 00,http://www.tui-group.com/dms/ir/20_annual_report_2009-10/pdfs/TUI_AR_2009-10_en/TUI_AR_2009-10_en.pdf


In [55]:
df_unlabeled = df_unlabeled.loc[0:20]

In [90]:
%%capture output 

# Scrape reports 

from pathlib import Path
import requests

df_unlabeled['CSR_Filename'] = ''

# Needed to prevent some pdfs from not downloading correctly
headers = {
    "User-Agent": "PostmanRuntime/7.20.1",
    "Accept": "*/*",
    "Cache-Control": "no-cache",
    "Postman-Token": "8eb5df70-4da6-4ba1-a9dd-e68880316cd9,30ac79fa-969b-4a24-8035-26ad1a2650e1",
    "Accept-Encoding": "gzip, deflate",
    "Connection": "keep-alive",
    "cache-control": "no-cache",
}

for index, row in df_unlabeled.iterrows():
    filename = row['CSR_Period_Relative'] + '_' + row['Identifier'] + '_' + '.pdf'
    pathname = Path('data/CSRs/' + filename)
    try:
        response = requests.get(row['CSR_URL'], headers=headers, verify=False)
        if 'application/pdf' in response.headers.get('content-type'): # check if link leads to a pdf file
            pathname.write_bytes(response.content)
            df_unlabeled.loc[index, 'CSR_Filename'] = filename
    except:
        df_unlabeled.loc[index, 'CSR_Filename'] = 'Error'

In [91]:
df_unlabeled

,Identifier,Company_Name,Country_of_Exchange,CSR_Period_Absolute,CSR_Period_Relative,CSR_Title,CSR_URL,CSR_Filename
0,BCE_py.TO,BCE Inc,Canada,FY2020,FY0,00 Purpose and Corporate Responsibility Report,https://www.bce.ca/responsibility/overview/2020-cr-report.pdf,FY0_BCE_py.TO_.pdf
1,ENB_pa.TO,Enbridge Inc,Canada,FY2020,FY0,00 Sustainability Report,https://www.enbridge.com/~/media/Enb/Documents/Reports/Sustainability%20Report%202020/Enbridge_SR_2020.pdf,FY0_ENB_pa.TO_.pdf
2,POW_pb.TO,Power Corporation of Canada,Canada,FY2019,FY0,09 Data Supplement,https://www.powercorporationcsr.com/media/uploads/reports/power-corporation-2019-data-supplement-final.pdf,FY0_POW_pb.TO_.pdf
3,PWF_pf.TO,Power Financial Corp,Canada,FY2018,FY0,Corporate Social Responsibility website,https://www.powerfinancialcsr.com/media/uploads/reports/pfc_-_csr_website_-_2018_update.pdf,FY0_PWF_pf.TO_.pdf
4,WN_pa.TO,George Weston Ltd,Canada,FY2019,FY0,Environment - Overview,http://www.weston.ca/en/Environment.aspx,
5,GWO_pf.TO,Great-West Lifeco Inc,Canada,FY2019,FY0,09 Public Accountability Statement,https://www.greatwestlifeco.com/content/dam/gwlco/documents/misc/canada-life-2019-public-accountability-statement.pdf,FY0_GWO_pf.TO_.pdf
6,MFC_pb.TO,Manulife Financial Corp,Canada,FY2020,FY0,Sustainability Report and Public Accountability Statement 00,https://www.manulife.com/content/dam/corporate/global/en/documents/pas/MFC_SR_PAS_2020.pdf,FY0_MFC_pb.TO_.pdf
7,BAM_pb.TO,NaN,Canada,FY2020,FY0,00 ESG REPORT,https://www.brookfield.com/sites/default/files/2021-06/2020_ESG_Report.pdf,
8,SLF_pc.TO,Sun Life Financial Inc,Canada,FY2020,FY0,Sustainability Report 00,https://www.sunlife.com/content/dam/sunlife/regional/global-marketing/documents/com/sustainability-report-2020-e.pdf,FY0_SLF_pc.TO_.pdf
9,FTS_pf.TO,Fortis Inc,Canada,FY2020,FY0,SUSTAINABILITY REPORT-0,https://sustainability.fortisinc.com/wp-content/uploads/2021/07/FORTIS_SUSTAINABILITY-REPORT-2021_DIGITAL.pdf,FY0_FTS_pf.TO_.pdf
